In [10]:
import pandas as pd

In [11]:
# data_path = "../data/data_investeringer.xlsx"
data_path = "../data/investeringer_datagrundlag.xlsx"
df = pd.read_excel(data_path)

In [ ]:
df

In [ ]:
# Filter the rows where "Kommune" is missing
missing_kommune_rows = df[df['Kommune'].isna()]

# Display the rows
print(missing_kommune_rows)

df = df[df['Kommune'].notna()]

In [ ]:
data_path = "../data/full_data.xlsx"
full_data = pd.read_excel(data_path)
full_data.head()

In [ ]:
# Group the data by 'Kommune' and check if there are any rows with a value in 'Priority'
grouped = full_data.groupby('Kommune')['Priority'].apply(lambda x: x.notna().any() and x.isin([1, 2, 3]).any())

# List of municipalities with at least one row with a priority value of 1, 2, or 3
municipalities_with_priority = grouped[grouped == True].index.tolist()

# List of municipalities with no rows having priority values
municipalities_without_priority = grouped[grouped == False].index.tolist()

# Display the results
print("Municipalities with at least one row with Priority (1, 2, or 3):")
print(municipalities_with_priority)

print("\nMunicipalities with no Priority values (1, 2, or 3):")
print(municipalities_without_priority)


In [ ]:
# Check that the lists are completely different
intersection = set(municipalities_with_priority).intersection(municipalities_without_priority)

# Display the lists and check for overlap
if not intersection:
    print("The lists are completely distinct.")
else:
    print("The following municipalities appear in both lists:")
    print(intersection)

In [ ]:
print(sum(full_data['Markedsværdi (DKK)']))
print(sum(df['Markedsværdi (DKK)']))

In [ ]:
import numpy as np

# Replace '-' with NaN (Fjerner dem, hvor der ikke er værdi. Det er fx to fra Odense)
df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].replace('-', np.nan)

# Remove any potential commas, spaces, or other non-numeric characters
df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].replace({',': '', ' ': ''}, regex=True)

# Convert the column to float
df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].astype(float)


In [ ]:
df.head()

In [ ]:
import pandas as pd

# Create the function to fill missing 'Type' based on the majority for each 'ISIN kode'
def fill_missing_type(df, min_rows=5, agree_threshold=0.8):
    def fill_type_for_group(group):
        # Count the missing values in 'Type' for this group
        missing_count = group['Type'].isna().sum()
        #print(f"ISIN kode: {group.name}, Missing 'Type' values: {missing_count}")
        
        # Get the count of each type in the group, excluding missing values
        type_counts = group['Type'].value_counts()
        
        # If there are no valid types in the group, skip this group
        if type_counts.empty:
            return group
        
        total_rows = len(group)
        most_common_type, most_common_count = type_counts.idxmax(), type_counts.max()
        
        # Check the condition: at least min_rows, and agreement should meet the threshold
        if total_rows >= min_rows and most_common_count / total_rows >= agree_threshold:
            # If conditions met, fill missing 'Type' with the most common type
            group['Type'] = group['Type'].fillna(most_common_type)
        
        return group

    # Group by 'ISIN kode' and apply the function to each group
    df = df.groupby('ISIN kode').apply(fill_type_for_group)

    return df

# Print the number of missing values in 'Type' before applying the function
missing_before = df['Type'].isna().sum()
print(f"Missing 'Type' values before: {missing_before}")

# Apply the function to fill missing 'Type' values
filled_df = fill_missing_type(df, min_rows=5, agree_threshold=0.80)

# Print the number of missing values in 'Type' after applying the function
missing_after = filled_df['Type'].isna().sum()
print(f"Missing 'Type' values after: {missing_after}")

filled_df.head()


In [ ]:
df_lav = df[df['Markedsværdi (DKK)'] < 100]
df_lav = df_lav[df_lav['Markedsværdi (DKK)'] >= 0]
df_lav

In [ ]:
sum(df_lav['Markedsværdi (DKK)'])

In [ ]:
df.value_counts('Kommune')

In [6]:
text = df[df['Kommune'] == 'Guldborgsund'].to_string(index=False)

In [ ]:
!pip install tiktoken

In [ ]:
import tiktoken
import pandas as pd

# Function to convert dataframe to text and count tokens
def dataframe_to_text(df):
    """
    This function converts a dataframe into a text format.
    You can modify it based on how you want to present the data.
    """
    text = df.to_string(index=False)  # Convert DataFrame to string (or JSON if needed)
    return text

# Function to count tokens using OpenAI's tokenizer (tiktoken)
def count_tokens(text, model="gpt-4"):
    """
    Counts the number of tokens in a given text based on the OpenAI model.
    """
    # Load the encoding for the specified model (gpt-3.5-turbo, gpt-4, etc.)
    encoding = tiktoken.encoding_for_model(model)
    
    # Encode the text into tokens
    tokens = encoding.encode(text)
    
    # Return the number of tokens
    return len(tokens)

# Convert dataframe to text
df_text = dataframe_to_text(df['Markedsværdi (DKK)'][df['Kommune'] == 'Guldborgsund'])

# Count tokens
num_tokens = count_tokens(df_text, model="gpt-4")

# print(f"DataFrame text:\n{df_text}")
print(f"\nNumber of tokens: {num_tokens}")

### Danwatch-check

In [2]:
import pandas as pd

In [3]:
data_path = "../data/datagrundlag_FN.xlsx"
df = pd.read_excel(data_path)

In [ ]:
df_fn = df[df['Problematisk ifølge:'] == 'FN']
sum(round(df_fn['Markedsværdi (DKK)'],5))

In [ ]:
len(df_fn)

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os 
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding


# Decrypt data using AES-CBC mode
def aes_decrypt(encrypted_data, key):
    encrypted_data = base64.b64decode(encrypted_data.encode())  # Decode Base64 to bytes
    iv = encrypted_data[:16]  # Extract the first 16 bytes as the IV
    ciphertext = encrypted_data[16:]

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    decrypted_padded_data = decryptor.update(ciphertext) + decryptor.finalize()

    # Remove padding
    unpadder = padding.PKCS7(128).unpadder()
    decrypted_data = unpadder.update(decrypted_padded_data) + unpadder.finalize()

    return decrypted_data.decode()

# Function to decrypt specified columns of the DataFrame using AES-CBC
def decrypt_dataframe(df, key, col_list):
    df_decrypted = df.copy()  # Create a copy of the DataFrame

    for col in col_list:
        if pd.api.types.is_string_dtype(df_decrypted[col]):
            # Decrypt string columns
            df_decrypted[col] = df_decrypted[col].apply(lambda x: aes_decrypt(x, key))
        else:
            # Convert to the original data type after decryption
            df_decrypted[col] = df_decrypted[col].apply(lambda x: aes_decrypt(str(x), key))

    return df_decrypted

# Only encrypt the necessary columns
col_list = ["Kommune", "ISIN kode", "Værdipapirets navn"]

# Fetch the base64 encoded key from environment variables
encoded_key = os.getenv("ENCRYPTION_KEY")

# Decode the key back to bytes for AES use
encryption_key = base64.b64decode(encoded_key)


# Create an SQLite engine
engine = create_engine("sqlite:///../src/investerings_database_encrypted_new.db")

# --- Optional: Decrypt the DataFrame for verification ---
df_retrieved = pd.read_sql("SELECT * FROM kommunale_regioner_investeringer", engine)
df_decrypted = decrypt_dataframe(df_retrieved, encryption_key, col_list)
print("Decrypted DataFrame:")
print(df_decrypted.head())


Decrypted DataFrame:
     Kommune     ISIN kode              Værdipapirets navn  \
0    Rødovre  AEA000801018     Abu Dhabi Islamic Bank PJSC   
1  Holstebro  AEA002001013                Aldar Properties   
2       Fanø  AEA002001013           Aldar Properties PJSC   
3       Fanø  AEE000301011           Emaar Properties PJSC   
4       Fanø  AEE000401019  Emirates Telecommunications Gr   

                      Udsteder  Markedsværdi (DKK)   Type  \
0  Abu Dhabi Islamic Bank PJSC         4420.591824  Aktie   
1             Aldar Properties        48912.517111  Aktie   
2         United Arab Emirates           56.605770  Aktie   
3         United Arab Emirates          396.240390  Aktie   
4         United Arab Emirates          396.240390  Aktie   

  Årsag til eksklusion OBS_Type  Priority Problematisk ifølge:  Sortlistet  \
0                 None     None       NaN                 None           0   
1                 None     None       NaN                 None           0   
2    

In [ ]:
# Step 1: Filter for rows where 'Priority' is 2 or 3
priority_2_or_3_df = df_decrypted[df_decrypted['Priority'].isin([2, 3])]

# Step 2: Get unique 'Kommune' values where 'Priority' is 2 or 3
kommuner_with_priority_2_or_3 = priority_2_or_3_df['Kommune'].unique()

# Step 3: Get all unique 'Kommune' values in the DataFrame
all_kommuner = df_decrypted['Kommune'].unique()

# Step 4: Find 'Kommune' values that don't have any 'Priority' of 2 or 3
kommuner_without_priority_2_or_3 = list(set(all_kommuner) - set(kommuner_with_priority_2_or_3))

# Print the results
print("Kommunes with Priority 2 or 3:", kommuner_with_priority_2_or_3)
print("Kommunes without Priority 2 or 3:", kommuner_without_priority_2_or_3)


In [ ]:
df_decrypted.columns

In [ ]:
import pandas as pd

# Assuming df_decrypted is your existing dataframe
# Step 1: Split the 'Problemkategori' column by ';' and expand them into separate rows
df_expanded = df_decrypted.assign(
    Problemkategori=df_decrypted['Problemkategori'].str.split(';')
).explode('Problemkategori')

# Step 2: Strip any extra spaces in 'Problemkategori'
df_expanded['Problemkategori'] = df_expanded['Problemkategori'].str.strip()

# Step 3: Grouping by 'Kommune' and 'Problemkategori', and aggregating
df_grouped = df_expanded.groupby(['Kommune', 'Problemkategori']).agg(
    Antal=('Problemkategori', 'size'),  # Counting occurrences
    Markedsværdi_sum=('Markedsværdi (DKK)', 'sum')  # Summing the values
).reset_index()

# Step 4: Rounding the 'Markedsværdi_sum' to 2 decimal places
df_grouped['Markedsværdi_sum'] = df_grouped['Markedsværdi_sum'].round(2)


# Saving the result to a CSV file
df_grouped.to_csv('investeringer_problemkategori.csv', index=False)

print(df_grouped)


In [ ]:
import pandas as pd

# Step 0: Filter the dataframe to keep only rows where 'Priority' is 2 or 3
df_filtered = df_decrypted[df_decrypted['Priority'].isin([2, 3])]

# Step 1: Split the 'Problemkategori' column by ';' and expand them into separate rows
df_expanded = df_filtered.assign(
    Problemkategori=df_filtered['Problemkategori'].str.split(';')
).explode('Problemkategori')

# Step 2: Strip any extra spaces in 'Problemkategori'
df_expanded['Problemkategori'] = df_expanded['Problemkategori'].str.strip()

# Step 3: Create a crosstab for 'Kommune' and 'Problemkategori'
df_crosstab = pd.crosstab(df_expanded['Kommune'], df_expanded['Problemkategori'])

# Step 4: Adding a 'Total' column that sums up all non-empty 'Problemkategori' counts
df_crosstab['Total'] = df_crosstab.sum(axis=1)

# Step 5: Get the total 'Markedsværdi (DKK)' sum for each 'Kommune' from the original df_decrypted
df_markedsværdi_sum = df_filtered.groupby('Kommune')['Markedsværdi (DKK)'].sum().reset_index()

df_result = df_crosstab.merge(df_markedsværdi_sum, on='Kommune')

df_result['Markedsværdi (DKK)'] = df_result['Markedsværdi (DKK)'].round(2)

# Step 6: Saving the result to a CSV file
df_result.to_csv('investeringer_problemkategori_kolonner.csv', index=False)

print(df_result)


In [7]:
#### Match med klima 

clima_list = df = pd.read_excel("../data/Climate-Action-100-Company-Assessments-Version-1.5.xlsx", sheet_name="Disclosure Assessments", header=8, skiprows=[9])
clima_list.head()

,Company name,ISIN,HQ Country,HQ Region,Sector cluster,Sector,Scope 3 cat,1.1 metric assessment,Unnamed: 8,Indicator 1 assessment,...,Unnamed: 72,Indicator 9,Unnamed: 74,10.1,Unnamed: 76,Sub indicator 10.1 assessment,10.2,Unnamed: 79,Sub indicator 10.2 assessment,Indicator 10 assessment
0,A.P. Moller (Maersk),DK0010244425,Denmark,Europe,Transportation,Shipping,No,Y,Not applicable,Y,...,NaN,Not assessed,NaN,Y,Y,Y,Y,N,Partial,Partial
1,Adelaide Brighton,AU000000ABC7,Australia,Oceania,Industrials,Cement,No,N,Not applicable,N,...,NaN,Not assessed,NaN,Y,Y,Y,Y,N,Partial,Partial
2,AES Corp.,US00130H1059,USA,North America,Energy,Electricity Utilities,No,Y,Not applicable,Y,...,NaN,Not assessed,NaN,Y,N,Partial,Y,Y,Y,Partial
3,AGL Energy Ltd.,AU000000AGL7,Australia,Oceania,Energy,Electricity Utilities,Yes (use of sold product from oil and gas dist...,Y,N,Partial,...,NaN,Not assessed,NaN,Y,Y,Y,Y,Y,Y,Y
4,Air France-KLM,FR0000031122,France,Europe,Transportation,Airlines,No,N,Not applicable,N,...,NaN,Not assessed,NaN,Y,Y,Y,N,N,N,Partial


In [12]:
# Merge the dataframes based on matching 'ISIN' from clima_list and 'ISIN kode' from decrypted_df
merged_df = pd.merge(df_decrypted, clima_list, left_on='ISIN kode', right_on='ISIN', how='inner')
merged_df.to_csv("samlet-data-sammenkørsel-med-Climate-Action-100-Company-Assessments.csv")


In [11]:
# Group by 'Kommune' to get the sum of 'Markedsværdi (DKK)' and the count of rows
grouped_df = merged_df.groupby('Kommune').agg(
    Markedsværdi_sum=('Markedsværdi (DKK)', 'sum'),  # Sum of 'Markedsværdi (DKK)'
    Antal=('Kommune', 'size')  # Count of rows (Antal) for each 'Kommune'
).reset_index()

grouped_df['Markedsværdi_sum'] = round(grouped_df['Markedsværdi_sum'], 2)
# Display the new dataframe
print(grouped_df)
grouped_df.to_csv('sammenkørsel-med-Climate-Action-100-Company-Assessments.csv')

print(len(grouped_df['Kommune'].unique()))


           Kommune  Markedsværdi_sum  Antal
0         Aabenraa        2896918.49    103
1          Aalborg        1697791.76     58
2           Aarhus       31796829.00     40
3           Assens        7154348.80    100
4         Ballerup        1552914.81     38
..             ...               ...    ...
72      Vallensbæk        1063143.38     62
73           Varde        4496635.46     60
74           Vejle          77989.47      4
75  Vesthimmerland        2321333.70      5
76          Viborg        6190912.22     77

[77 rows x 3 columns]
77
